# Reddit Posts Data Processing

[2022 Sentiment Analysis of Reddit Posts and BTC](https://www.kaggle.com/code/joehardin369/2022-sentiment-analysis-of-reddit-posts-and-btc)


This fils is used to extract the subset from the whole dataset, to get the timeperiod from 2022-1-1 to 2022-6-30.
At the same time, it is used to drop the unuseful columns and transform the timestamp to the datetime.

In [73]:
import pandas as pd
import numpy as np
import datetime
import os
from datetime import date
folder_path="./data/Reddit/"

In [74]:
# Extract the subset within the specific timeperiod
def is_within_range(input_date):
    start_date=date(2022,1,1)
    end_date=date(2022,6,30)
    return start_date <= input_date <= end_date

In [75]:
# Read the csv files
Merge_data=pd.DataFrame()
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # read the csv filename
        reddit_posts=pd.read_csv(os.path.join(folder_path, filename))
        # create the new column called "date_created"
        reddit_posts['date_created']=(reddit_posts['created']).apply(lambda x: datetime.datetime.fromtimestamp(x).date())

        # Extract the data from 2022-1-1 to 2022-6-30
        subset_set=[row for i,row in reddit_posts.iterrows() if is_within_range(row.loc['date_created'])]
        subreddits= pd.DataFrame(subset_set)

        # Determine the accurate time
        subreddits["datetime"]= subreddits['created'].apply(lambda x: datetime.datetime.fromtimestamp(x))

        # apply the floor function to the data, get the hourly data
        subreddits['datetime']=subreddits['datetime'].dt.floor('H')
        subreddits['datetime']=subreddits['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
        # Drop the unuseful columns, don't keep the title
        columns_keep=['datetime','created','date_created','upvote_ratio','score','total_awards_received','num_comments','selftext']
        columns_delete=list(subreddits.columns.values)
        for keep in columns_keep:
            columns_delete.remove(keep)
        subreddits.drop(columns_delete, axis=1, inplace=True)
        subreddits = subreddits[subreddits['selftext'].str.len()>=10]
        subreddits.drop(columns=["date_created","created"],inplace=True)
        Merge_data=pd.concat([Merge_data, subreddits])

In [76]:
Merge_data.head(10)

,upvote_ratio,score,total_awards_received,num_comments,selftext,datetime
0,0.33,0,0,14,"I recently caught a red herring, and you can d...",2022-01-01 00:00:00
1,0.24,0,0,22,I noticed that almost everyone in the crypto c...,2022-01-01 00:00:00
5,1.00,13,0,41,My records show I still had some BCHA on my Me...,2022-01-01 02:00:00
9,0.73,24,0,188,Merry Christmas and a Happy New Year. \n\nPS: ...,2022-01-01 05:00:00
11,0.33,0,0,7,If you look at Bitcoin satoshies! There's $2....,2022-01-01 07:00:00
15,0.94,28,0,22,[Gaze.cash](https://gaze.cash/) is a video co...,2022-01-01 11:00:00
20,0.58,6,0,58,I started to look more projects came to know t...,2022-01-01 11:00:00
27,0.69,10,0,13,\n\n[View Poll](https://www.reddit.com/poll/rt...,2022-01-01 14:00:00
39,0.87,16,0,4,Starting a new project soon... An African Cryp...,2022-01-01 18:00:00
44,0.84,17,0,9,"My plan will be the following, due to the low ...",2022-01-01 19:00:00


In [77]:
len(Merge_data)

31118

In [79]:
Merge_data.to_csv(os.path.join(folder_path,"Reddit.csv"), index=False)